# **AI Applications Project**

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
# import datasets
listings_url = 'https://data.insideairbnb.com/united-states/la/new-orleans/2025-09-11/data/listings.csv.gz'
calendar_url = 'https://data.insideairbnb.com/united-states/la/new-orleans/2025-09-11/data/calendar.csv.gz'
reviews_url = 'https://data.insideairbnb.com/united-states/la/new-orleans/2025-09-11/data/reviews.csv.gz'

# Load the datasets into DataFrames
listings_df = pd.read_csv(listings_url, compression='gzip')
calendar_df = pd.read_csv(calendar_url, compression='gzip')
reviews_df = pd.read_csv(reviews_url, compression='gzip')



In [ ]:
calendar_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,19091,2025-09-11,t,NaN,NaN,1,365
1,19091,2025-09-12,t,NaN,NaN,1,365
2,19091,2025-09-13,f,NaN,NaN,1,365
3,19091,2025-09-14,f,NaN,NaN,1,365
4,19091,2025-09-15,t,NaN,NaN,1,365


In [ ]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2717060 entries, 0 to 2717059
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           float64
 4   adjusted_price  float64
 5   minimum_nights  int64  
 6   maximum_nights  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 145.1+ MB


In [ ]:
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,19091,35930,2010-04-16,97343,Melissa,John was extremely accommodating to our schedu...
1,19091,45900,2010-05-24,99114,Casey,John was a great host and made sure that I cou...
2,19091,61668,2010-07-06,128406,Sherian,My husband and I stayed with John last month a...
3,19091,96538,2010-09-13,228880,Anthony,John is a great host for your time in New Orle...
4,19091,108793,2010-09-30,214223,Candace,john is a fantastic host. the place is perfe...


In [ ]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489253 entries, 0 to 489252
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     489253 non-null  int64 
 1   id             489253 non-null  int64 
 2   date           489253 non-null  object
 3   reviewer_id    489253 non-null  int64 
 4   reviewer_name  489252 non-null  object
 5   comments       489100 non-null  object
dtypes: int64(3), object(3)
memory usage: 22.4+ MB


In [ ]:
listings_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,19091,https://www.airbnb.com/rooms/19091,20250911031349,2025-09-11,city scrape,Fully Furnished Cozy Apartment,This cozy space is a wonder of post-Katrina re...,"NorthWest Carrollton, where this apartment is ...",https://a0.muscache.com/pictures/a5fdfd3b-5d81...,72880,...,4.98,4.68,4.91,"25-NSTR-11772, 25-OSTR-11151",f,1,1,0,0,2.72
1,71624,https://www.airbnb.com/rooms/71624,20250911031349,2025-09-11,city scrape,Ravenwood Manor (Historic Bywater),You are in for a very unique stay in a neighbo...,"Ours is a neighborhood made up of artists, mus...",https://a0.muscache.com/pictures/37488608/8cc1...,367223,...,4.98,4.84,4.89,"21-RSTR-18609, 22-OSTR-20720",f,1,1,0,0,1.78
2,74498,https://www.airbnb.com/rooms/74498,20250911031349,2025-09-11,city scrape,Maison Marais 1,Maison Marais 1 is a stylish one-bedroom apart...,"This is a vibrant, diverse neighborhood of mos...",https://a0.muscache.com/pictures/73591836/a1f5...,391462,...,4.96,4.67,4.85,23-XSTR-13082,f,3,3,0,0,3.75
3,79536,https://www.airbnb.com/rooms/79536,20250911031349,2025-09-11,previous scrape,FeelAtHomeInNewOrleans-PrivateApt,"Safe, clean & functional!! Front porch and gat...","I love that it's down to earth, welcoming, his...",https://a0.muscache.com/pictures/f1037ebf-1573...,428362,...,4.98,4.56,4.87,"23-NSTR-15067, 24-OSTR-20882",t,1,1,0,0,4.57
4,79609,https://www.airbnb.com/rooms/79609,20250911031349,2025-09-11,city scrape,Bywater's Original & Finest,The Art of Hospitality,"First settled in 1806, Bywater is known for it...",https://a0.muscache.com/pictures/5c5da2eb-7434...,428909,...,4.99,4.84,4.94,"23-ISTR-02823, 23-OSTR-02823",f,1,1,0,0,2.86


In [ ]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7444 entries, 0 to 7443
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            7444 non-null   int64  
 1   listing_url                                   7444 non-null   object 
 2   scrape_id                                     7444 non-null   int64  
 3   last_scraped                                  7444 non-null   object 
 4   source                                        7444 non-null   object 
 5   name                                          7444 non-null   object 
 6   description                                   7373 non-null   object 
 7   neighborhood_overview                         4429 non-null   object 
 8   picture_url                                   7444 non-null   object 
 9   host_id                                       7444 non-null   i

### Identifying Missing Values

In [ ]:
print("Missing values in listings_df:")
display(listings_df.isnull().sum()[listings_df.isnull().sum() > 0])

print("\nMissing values in calendar_df:")
display(calendar_df.isnull().sum()[calendar_df.isnull().sum() > 0])

print("\nMissing values in reviews_df:")
display(reviews_df.isnull().sum()[reviews_df.isnull().sum() > 0])

# Handle missing values in listings_df
# Drop columns with a high percentage of missing values if they exist
cols_to_drop = ['neighbourhood_group_cleansed', 'calendar_updated']
existing_cols_to_drop = [col for col in cols_to_drop if col in listings_df.columns]
if existing_cols_to_drop:
    listings_df.drop(columns=existing_cols_to_drop, inplace=True)


# Impute numerical columns with the mean
numerical_cols = listings_df.select_dtypes(include=np.number).columns
for col in numerical_cols:
    if listings_df[col].isnull().sum() > 0:
        listings_df[col].fillna(listings_df[col].mean(), inplace=True)

# Impute categorical columns with the mode
categorical_cols = listings_df.select_dtypes(include='object').columns
for col in categorical_cols:
    if listings_df[col].isnull().sum() > 0:
        listings_df[col].fillna(listings_df[col].mode()[0], inplace=True)

# Handle missing values in calendar_df
# The 'price' and 'adjusted_price' columns have all missing values and will be handled during formatting.

# Handle missing values in reviews_df
# Impute 'reviewer_name' with a placeholder
reviews_df['reviewer_name'].fillna('Anonymous', inplace=True)
# Impute 'comments' with an empty string
reviews_df['comments'].fillna('', inplace=True)

print("\nMissing values after handling in listings_df:")
display(listings_df.isnull().sum()[listings_df.isnull().sum() > 0])

print("\nMissing values after handling in calendar_df:")
display(calendar_df.isnull().sum()[calendar_df.isnull().sum() > 0])

print("\nMissing values after handling in reviews_df:")
display(reviews_df.isnull().sum()[reviews_df.isnull().sum() > 0])

Missing values in listings_df:


,0
description,71
neighborhood_overview,3015
host_name,2
host_since,2
host_location,2341
host_about,3285
host_response_time,1596
host_response_rate,1596
host_acceptance_rate,1155
host_is_superhost,268



Missing values in calendar_df:


,0
price,2717060
adjusted_price,2717060



Missing values in reviews_df:


...

,0



Missing values after handling in calendar_df:


,0
price,2717060
adjusted_price,2717060



Missing values after handling in reviews_df:


,0


In [ ]:
print("Descriptive statistics for listings_df:")
display(listings_df.describe())

print("\nDescriptive statistics for calendar_df:")
display(calendar_df.describe())

print("\nDescriptive statistics for reviews_df:")

Descriptive statistics for listings_df:


,id,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,7.444000e+03,7.444000e+03,7.444000e+03,7442.000000,7442.000000,0.0,7444.000000,7444.000000,7444.000000,6254.000000,...,6154.000000,6153.000000,6154.000000,6153.000000,6153.000000,7444.000000,7444.000000,7444.000000,7444.000000,6154.000000
mean,6.091228e+17,2.025091e+13,2.415269e+08,104.547299,167.441682,NaN,29.957996,-90.072732,5.104782,1.798689,...,4.797970,4.875573,4.860392,4.739085,4.720418,15.132456,13.065825,1.093632,0.002552,1.559079
std,5.700130e+17,1.367279e+00,2.160182e+08,465.705181,748.793782,NaN,0.024044,0.031311,3.188474,1.326395,...,0.332827,0.305000,0.328026,0.347593,0.379996,23.581411,22.142376,3.234025,0.075965,1.712353
min,1.909100e+04,2.025091e+13,3.326500e+04,1.000000,1.000000,NaN,29.897042,-90.135521,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,3.363779e+07,2.025091e+13,4.673728e+07,1.000000,2.000000,NaN,29.942357,-90.087440,2.000000,1.000000,...,4.750000,4.880000,4.860000,4.630000,4.670000,1.000000,1.000000,0.000000,0.000000,0.420000
50%,6.812117e+17,2.025091e+13,1.627343e+08,4.000000,6.000000,NaN,29.955531,-90.072818,4.000000,1.500000,...,4.890000,4.960000,4.960000,4.840000,4.810000,3.000000,2.000000,0.000000,0.000000,1.160000
75%,1.163053e+18,2.025091e+13,4.429315e+08,23.000000,35.000000,NaN,29.968886,-90.062647,6.000000,2.000000,...,4.990000,5.000000,5.000000,4.960000,4.910000,16.000000,12.000000,0.000000,0.000000,2.230000
max,1.506566e+18,2.025091e+13,7.138135e+08,3883.000000,9658.000000,NaN,30.161040,-89.737090,16.000000,44.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,84.000000,83.000000,23.000000,3.000000,42.710000



Descriptive statistics for calendar_df:


,listing_id,price,adjusted_price,minimum_nights,maximum_nights
count,2.717060e+06,0.0,0.0,2.717060e+06,2.717060e+06
mean,6.091228e+17,NaN,NaN,1.603107e+01,3.173965e+06
std,5.699748e+17,NaN,NaN,2.498753e+01,8.249011e+07
min,1.909100e+04,NaN,NaN,1.000000e+00,1.000000e+00
25%,3.363779e+07,NaN,NaN,2.000000e+00,3.650000e+02
50%,6.812117e+17,NaN,NaN,3.000000e+00,3.650000e+02
75%,1.163053e+18,NaN,NaN,3.000000e+01,1.125000e+03
max,1.506566e+18,NaN,NaN,1.125000e+03,2.147484e+09



Descriptive statistics for reviews_df:


In [ ]:
print("Correlation matrix for listings_df:")
display(listings_df.select_dtypes(include=np.number).corr())

print("\nCorrelation matrix for calendar_df:")
display(calendar_df.select_dtypes(include=np.number).drop(columns=['listing_id']).corr())

print("\nCorrelation matrix for reviews_df:")
display(reviews_df.select_dtypes(include=np.number).drop(columns=['listing_id', 'id', 'reviewer_id']).corr())

Correlation matrix for listings_df:


,id,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,1.000000,NaN,0.526187,0.106134,0.106228,NaN,-0.013694,0.067666,0.144981,0.110884,...,-0.087271,-0.175047,-0.159245,-0.091164,-0.133571,0.296802,0.262931,0.138530,-0.014852,0.002001
scrape_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
host_id,0.526187,NaN,1.000000,0.188672,0.169663,NaN,-0.008699,0.086145,0.157838,0.080033,...,-0.140139,-0.198818,-0.206976,-0.118011,-0.188878,0.303184,0.246882,0.159398,-0.022997,-0.006807
host_listings_count,0.106134,NaN,0.188672,1.000000,0.945269,NaN,-0.044537,0.029936,-0.000773,-0.041474,...,-0.021339,-0.093021,-0.098801,-0.035442,-0.117337,0.301251,0.099032,0.048225,-0.007306,-0.070528
host_total_listings_count,0.106228,NaN,0.169663,0.945269,1.000000,NaN,-0.033149,0.037044,0.020457,-0.029676,...,-0.032039,-0.095808,-0.101931,-0.045274,-0.128203,0.291256,0.075839,0.030784,-0.007296,-0.060423
neighbourhood_group_cleansed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
latitude,-0.013694,NaN,-0.008699,-0.044537,-0.033149,NaN,1.000000,0.514588,-0.005098,-0.023287,...,-0.010319,0.013653,0.004202,-0.082141,-0.005029,-0.118393,-0.117997,0.009934,-0.001500,-0.059951
longitude,0.067666,NaN,0.086145,0.029936,0.037044,NaN,0.514588,1.000000,0.062335,0.032510,...,-0.014038,-0.030304,-0.035851,-0.057271,-0.031646,0.033588,0.021485,0.053890,-0.005076,-0.023155
accommodates,0.144981,NaN,0.157838,-0.000773,0.020457,NaN,-0.005098,0.062335,1.000000,0.687891,...,-0.027254,-0.053922,-0.024696,-0.091094,-0.037311,0.172063,0.187272,-0.095037,-0.028839,-0.028414
bathrooms,0.110884,NaN,0.080033,-0.041474,-0.029676,NaN,-0.023287,0.032510,0.687891,1.000000,...,0.057779,0.009357,0.048737,-0.019431,0.055526,0.113835,0.130173,-0.034829,0.010301,-0.015975



Correlation matrix for calendar_df:


,price,adjusted_price,minimum_nights,maximum_nights
price,NaN,NaN,NaN,NaN
adjusted_price,NaN,NaN,NaN,NaN
minimum_nights,NaN,NaN,1.000000,-0.022326
maximum_nights,NaN,NaN,-0.022326,1.000000



Correlation matrix for reviews_df:


""
